In [51]:
from tqdm.notebook import tqdm

import numpy as np
import pandas as pd

import xgboost as xgb

from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV

from sklearn.metrics import mean_squared_error, classification_report, log_loss

In [26]:
df = pd.read_csv("mustapha_train.csv")
df

,sub/body,urls/body,images/body,salutaions&designation,ccs,bcced,images,urls,salutations,designation,...,org_api,org_youth4work,org_diu,org_zoovsupport,org_paypalexchanges,org_zen2makemytrip,org_agencenavigo,org_asus,org_nitkkr,org_length
0,0.001839,0.001096,0.000157,0,0,0,4,28,0,1,...,0,0,0,0,0,0,0,0,0,16
1,0.002308,0.001606,0.000301,0,0,0,6,32,0,0,...,0,0,0,0,0,0,0,0,0,4
2,5.250000,0.000000,0.000000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
3,0.000538,0.001771,0.001118,0,0,0,108,171,0,0,...,0,0,0,0,0,0,0,0,0,11
4,0.000851,0.001745,0.000210,0,0,0,20,166,0,0,...,0,0,0,0,0,0,0,0,0,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39666,0.000712,0.000690,0.000245,0,0,0,11,31,0,0,...,0,0,0,0,0,0,0,0,0,3
39667,0.079057,0.000000,0.000000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
39668,0.049037,0.000000,0.000000,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,5
39669,0.000258,0.001465,0.000291,1,0,0,27,136,1,1,...,0,0,0,0,0,0,0,0,0,1


In [18]:
'updates' in df_train.columns

True

In [19]:
df_test = pd.read_csv("mustapha_test.csv")
df_test

,ccs,bcced,images,urls,salutations,designation,chars_in_subject,chars_in_body,date_dayweek,date_hour,...,org_api,org_youth4work,org_diu,org_zoovsupport,org_paypalexchanges,org_zen2makemytrip,org_agencenavigo,org_asus,org_nitkkr,org_length
0,0,0,7,56,0,0,67.0,36243,4,8,...,0,0,0,0,0,0,0,0,0,7
1,0,0,5,33,0,0,27.0,27015,0,14,...,0,0,0,0,0,0,0,0,0,6
2,0,0,0,2,1,0,22.0,788,1,10,...,0,0,0,0,0,0,0,0,0,5
3,0,0,8,53,0,0,79.0,39504,4,9,...,0,0,0,0,0,0,0,0,0,7
4,0,0,0,0,0,0,24.0,178773,1,1,...,0,0,0,0,0,0,0,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16997,1,0,0,8,0,0,73.0,2372,2,12,...,0,0,0,0,0,0,0,0,0,6
16998,0,0,7,27,0,0,39.0,13674,5,12,...,0,0,0,0,0,0,0,0,0,5
16999,0,0,0,0,0,0,25.0,19845,4,14,...,0,0,0,0,0,0,0,0,0,5
17000,0,0,1,6,0,0,39.0,17565,3,9,...,0,0,0,0,0,0,0,0,0,12


In [20]:
'updates' in df_test.columns

False

In [29]:
labels = ['updates', 'personal', 'promotions', 
          'forums', 'purchases', 'travel','spam', 'social']

X_full = df.drop(columns=labels)
y = df[labels]

X_train, X_test, y_train_full, y_test_full = train_test_split(
    X_full, y, test_size=0.25, random_state=1, stratify=y)

X_train, X_val, y_train_full, y_val_full = train_test_split(
    X_train, y_train_full, test_size=0.20, random_state=1, stratify=y_train_full)

In [32]:
dic_models = dict()

for label in tqdm(labels):
    print(f"-------- For {label}: --------")
    
    y_train = y_train_full[label]
    y_test = y_test_full[label]
    
    dic_models[label] = xgb.XGBClassifier(objective='binary:logistic')
    dic_models[label].fit(X_train, y_train)
    
    y_preds = dic_models[label].predict(X_test)
    
    print(classification_report(y_test, y_preds))
    print(f"Log loss: {log_loss(y_test, y_preds)}")
    print("\n\n")

  0%|          | 0/8 [00:00<?, ?it/s]

-------- For updates: --------
[17:25:02] WARNING: /Users/runner/miniforge3/conda-bld/xgboost_1607604592557/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
              precision    recall  f1-score   support

           0       0.93      0.93      0.93      6324
           1       0.88      0.88      0.88      3594

    accuracy                           0.91      9918
   macro avg       0.91      0.91      0.91      9918
weighted avg       0.91      0.91      0.91      9918

Log loss: 3.0193043519348923
-------- For personal: --------


/Users/Tony/opt/anaconda3/envs/ml/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:25:16] WARNING: /Users/runner/miniforge3/conda-bld/xgboost_1607604592557/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
              precision    recall  f1-score   support

           0       0.94      0.91      0.93      1888
           1       0.98      0.99      0.98      8030

    accuracy                           0.97      9918
   macro avg       0.96      0.95      0.96      9918
weighted avg       0.97      0.97      0.97      9918

Log loss: 0.9193755296471184
-------- For promotions: --------


/Users/Tony/opt/anaconda3/envs/ml/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

[17:26:13] WARNING: /Users/runner/miniforge3/conda-bld/xgboost_1607604592557/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      9893
           1       0.94      0.64      0.76        25

    accuracy                           1.00      9918
   macro avg       0.97      0.82      0.88      9918
weighted avg       1.00      1.00      1.00      9918

Log loss: 0.03482441657053306
-------- For spam: --------


/Users/Tony/opt/anaconda3/envs/ml/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:26:26] WARNING: /Users/runner/miniforge3/conda-bld/xgboost_1607604592557/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      9880
           1       0.92      0.63      0.75        38

    accuracy                           1.00      9918
   macro avg       0.96      0.82      0.87      9918
weighted avg       1.00      1.00      1.00      9918

Log loss: 0.05571909876118584
-------- For social: --------


/Users/Tony/opt/anaconda3/envs/ml/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:26:36] WARNING: /Users/runner/miniforge3/conda-bld/xgboost_1607604592557/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      8917
           1       0.97      0.95      0.96      1001

    accuracy                           0.99      9918
   macro avg       0.98      0.97      0.98      9918
weighted avg       0.99      0.99      0.99      9918

Log loss: 0.271632077790995


# Per day for personnal and updates

## GridSearch for one model

In [37]:
labels = ['updates', 'personal', 'promotions', 
          'forums', 'purchases', 'travel','spam', 'social']
current_labels = ['updates', 'personal']

For Monday and `updates`:

In [38]:
df_weekday = df[df["date_dayweek"]==0]

X_full = df_weekday.drop(columns=labels)
y = df_weekday[current_labels]

X_train, X_test, y_train_full, y_test_full = train_test_split(
    X_full, y, test_size=0.25, random_state=1, stratify=y)

X_train, X_val, y_train_full, y_val_full = train_test_split(
    X_train, y_train_full, test_size=0.20, random_state=1, stratify=y_train_full)


y_train = y_train_full['updates']
y_test = y_test_full['updates']

In [59]:
# A parameter grid for XGBoost
params = {
        'n_estimators': [100],
        'min_child_weight': [1, 5],
        'gamma': [1],
        'subsample': [0,4, 0.6, 0.8],
        'colsample_bytree': [0.8],
        'max_depth': range(2, 12)
        }

# It's more efficient to use parallelization for GridSearch so we set n_jobs=1:
clf = xgb.XGBClassifier(n_jobs=1, verbosity=0, use_label_encoder=False) 

grid = RandomizedSearchCV(
    clf,
    params,
    n_iter=50,
    n_jobs=4,  # -1 means using all processors
    scoring="neg_log_loss",
    cv=3,
    refit=True,
    verbose=1)

grid.fit(X_train, y_train, verbose=3)

model = grid.best_estimator_

# Save the optimal hyperparameters
with open('XGBClassifier_params_0_updates.json', "w") as f:  
    json.dump(grid.best_params_, f)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
/Users/Tony/opt/anaconda3/envs/ml/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   13.4s
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:   53.0s finished


In [60]:
grid.best_params_

{'subsample': 0.8,
 'n_estimators': 100,
 'min_child_weight': 1,
 'max_depth': 11,
 'gamma': 1,
 'colsample_bytree': 0.8}

In [61]:
grid.best_score_

-0.3406412905170766

In [62]:
model

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=1, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=11,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=1, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=0.8,
              tree_method='exact', use_label_encoder=False,
              validate_parameters=1, verbosity=0)

## For all models

In [33]:
labels = ['updates', 'personal', 'promotions', 
          'forums', 'purchases', 'travel','spam', 'social']
current_labels = ['updates', 'personal']

In [35]:
X_train.date_dayweek

6728     1
1298     1
3289     3
25202    4
296      2
        ..
8895     1
3236     1
12222    0
29832    1
29065    5
Name: date_dayweek, Length: 23802, dtype: int64

In [36]:
dic_models = dict()

for weekday in range(7):
    df_weekday = df[df["date_dayweek"]==weekday]
    
    for label in current_labels:
        key = (weekday, label)
        
        print(f"-------- For {weekday} and {label}: --------")
        
        X_full = df_weekday.drop(columns=labels)
        y = df_weekday[current_labels]

        X_train, X_test, y_train_full, y_test_full = train_test_split(
            X_full, y, test_size=0.25, random_state=1, stratify=y)

        X_train, X_val, y_train_full, y_val_full = train_test_split(
            X_train, y_train_full, test_size=0.20, random_state=1, stratify=y_train_full)

        
        y_train = y_train_full[label]
        y_test = y_test_full[label]

        dic_models[key] = xgb.XGBClassifier(objective='binary:logistic')
        dic_models[key].fit(X_train, y_train)

        y_preds = dic_models[key].predict(X_test)

        print(classification_report(y_test, y_preds))
        print(f"Log loss: {log_loss(y_test, y_preds)}")
        print("\n\n")

-------- For 0 and updates: --------
[17:33:46] WARNING: /Users/runner/miniforge3/conda-bld/xgboost_1607604592557/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/Users/Tony/opt/anaconda3/envs/ml/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


              precision    recall  f1-score   support

           0       0.91      0.93      0.92       534
           1       0.88      0.84      0.86       309

    accuracy                           0.90       843
   macro avg       0.89      0.89      0.89       843
weighted avg       0.90      0.90      0.90       843

Log loss: 3.5235638850145152



-------- For 0 and personal: --------
[17:33:47] WARNING: /Users/runner/miniforge3/conda-bld/xgboost_1607604592557/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/Users/Tony/opt/anaconda3/envs/ml/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


              precision    recall  f1-score   support

           0       0.92      0.87      0.90       134
           1       0.98      0.99      0.98       709

    accuracy                           0.97       843
   macro avg       0.95      0.93      0.94       843
weighted avg       0.97      0.97      0.97       843

Log loss: 1.1062402797377255



-------- For 1 and updates: --------


...

[17:34:12] WARNING: /Users/runner/miniforge3/conda-bld/xgboost_1607604592557/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
              precision    recall  f1-score   support

           0       0.93      0.88      0.91       274
           1       0.98      0.99      0.98      1294

    accuracy                           0.97      1568
   macro avg       0.95      0.93      0.94      1568
weighted avg       0.97      0.97      0.97      1568

Log loss: 1.1013803615199629



-------- For 6 and updates: --------
[17:34:14] WARNING: /Users/runner/miniforge3/conda-bld/xgboost_1607604592557/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore 

/Users/Tony/opt/anaconda3/envs/ml/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


              precision    recall  f1-score   support

           0       0.91      0.92      0.92       613
           1       0.90      0.88      0.89       470

    accuracy                           0.91      1083
   macro avg       0.90      0.90      0.90      1083
weighted avg       0.91      0.91      0.91      1083

Log loss: 3.2529942505633516



-------- For 6 and personal: --------
[17:34:16] WARNING: /Users/runner/miniforge3/conda-bld/xgboost_1607604592557/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/Users/Tony/opt/anaconda3/envs/ml/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


              precision    recall  f1-score   support

           0       0.93      0.91      0.92       225
           1       0.98      0.98      0.98       858

    accuracy                           0.97      1083
   macro avg       0.95      0.94      0.95      1083
weighted avg       0.97      0.97      0.97      1083

Log loss: 1.1481188751272573



